# Assignment 1

**Handin in Peergrade**: *Wednesday*, March 21, 2018, 23:59<br>
**Peergrading deadline**: *Sunday*, March 25, 2018, 23:59<br>
**Peergrading feedback deadline**: *Wednesday*, March 28, 2018, 23:59

[**Questions**](https://github.com/abjer/tsds/issues)

## Problems

### Week 1

>**Ex 1.1.12**: Just before, you learned about a `dict`, so now you're ready for a `defaultdict`.
1. What is a `defaultdict`? How would you say it is different from a normal Python `dict`?
2. Write some code that takes a list of tuples:

>        l = [("a", 1), ("b", 3), ("a", None), ("c", False), ("b", True), ("a", None)]

>     And produces a `defaultdict` object

>        defaultdict(<type 'list'>, {'a': [1, None, None], 'c': [False], 'b': [3, True]})

>*Hint: you can import `defaultdict` from `collections`*

>**Ex 1.1.14**: Take two lists `a = list("justreadtheinstructions")` and `b = list("ofcourseistillloveyou")` and
1. get the `set` of characters that exist in both `a` and `b` (intersection),
2. get the `set` of characters that exist in either `a` or `b` (union), and
3. compute the [Jaccard similarity](https://en.wikipedia.org/wiki/Jaccard_index) between the distinct elements in `a` and `b`.

>*Hint: use the `set` function to get a `set`-type object of distinct elements from a list*

>**Ex. 1.2.2**: Use [`requests`](https://www.google.dk/search?q=python+requests+get+json&gws_rd=cr&ei=M5OdWaewD8Ti6AS54J24Bg), or another Python module, to store **[this data](https://www.reddit.com/r/gameofthrones/.json)** in JSON format in a new variable `data`. Show that the type of `data` is `dict`.

>**Ex 1.2.5**: Each post has a key that indicates whether it is a spoiler or not. Write two `for` loops (or list comprehensions for extra street credits):
>1. One that counts the number of spoilers.
>2. Another that only prints headlines that aren't spoilers.

### Week 2

Use the code provided in the solutions to obtain dataset `data`.

In [4]:
import re

import pandas as pd
import requests

r = requests.get('http://data.kk.dk/dataset/faste-trafiktaellinger')
html = r.text

re_url = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
urls = re.findall(re_url, html)

urls_w_data = [u for u in urls if u.endswith('.xlsx')]

data_sets = [pd.read_excel(u, skiprows=10) for u in urls_w_data]

data_raw = pd.concat(data_sets)

data_idx = data_raw.reset_index(drop=True)
data_idx.drop('Spor', axis=1, inplace=True)

dk_to_uk = {'Vejnavn':'road_name',
            '(UTM32)':'UTM32_east',
            '(UTM32).1':'UTM32_north',
            'Dato':'date',
            'Vej-Id':'road_id'}

data_long = data_idx\
                .rename(columns={'index':'year'})\
                .rename(columns=dk_to_uk)
        
data_long = data_long.copy()

del data_sets, data_raw, data_idx        

data_long['total'] = data_long.road_id.str[-1] == 'T'

data_long = data_long[data_long.total]

spatial_columns = ['road_name', 'UTM32_north', 'UTM32_east']
data_geo = data_long[spatial_columns].copy()

data_geo.drop_duplicates(inplace=True)

id_columns = ['road_name', 'date']
hour_columns = [c for c in data_long.columns if 'kl.' in c]

data = pd.melt(data_long, 
               id_vars=id_columns, 
               value_vars=hour_columns,
               var_name='hour_period',
               value_name='traffic')\
         .copy()

del data_long

data.road_name = data.road_name.astype('category')


data['hour'] = data.hour_period.str[3:5]
time_str = data.date + '-' + data.hour              
data['time'] = pd.to_datetime(time_str, format='%d.%m.%Y-%H')

data.drop(['hour', 'date', 'hour_period'], 1, inplace=True)
data['week_day'] = data.time.dt.weekday

> **Ex. 2b.5.2**: Which road has the most average traffic?

> *Hint: Start with a `groupby('road_name')` operation on `data`.*

> **Ex. 2b.5.3**: Compute annual, average road traffic during day hours (9-17). Which station had the least traffic in 2013? Which station has seen highest growth in traffic from 2013 to 2014?

### Week 4

> **Ex: 4.2.2**: Create a `networkx.Graph` that represents each partnership between characters, as an edge. Print the number of nodes, edges and average degree of the resulting network.

> **Ex: 4.2.3**: Plot the degree distribution of your character network. What type of random network does it resemble?

> **Ex: 4.2.4**: Create three bar charts, side-by-side, each with the 10 most high-degree characters one of the classes. Make sure that each subfigure has the same y-axis (control with `plt.ylim` or use `plt.subplots` with parameter `sharey=True`).

> **Ex: 4.2.5**: Do the same, but for the highest [betweenness centrality](https://networkx.github.io/documentation/networkx-1.10/reference/generated/networkx.algorithms.centrality.betweenness_centrality.html) (BC). Again, make sure the y-axes are shared. When you comment on the result:
* Explain briefly what the BC measures (and why it takes some time to compute). An intuitive explanation is also great!
* Interpret, if you can, what the differences you see between the distribution of degree and BC means for the different classes.

### Week 5

> **Ex. 5.1.2**: Implement the modularity function. Write a Python function that takes as input an adjacency matrix and a label vector, and returns the modularity. Compute and print the modularity for the ones given below.

In [119]:
import numpy as np

A = np.array([
    [0, 1, 1, 0, 0, 0],
    [1, 0, 1, 0, 0, 0],
    [1, 1, 0, 1, 0, 0],
    [0, 0, 1, 0, 1, 1],
    [0, 0, 0, 1, 0, 1],
    [0, 0, 0, 1, 1, 0],
])

c = [0, 0, 0, 0, 1, 1]

def modularity(A, c):
    """Compute modularity for a labeled network.
    
    Input
    -----
        A : numpy.array
            Adjacency matrix. (N, N) square matrix.
        c : list of ints
            Community labels. Length N.
    
    Output
    ------
        out : float
    """
    # Your beautiful code goes here

Now let's work with some real data. Whip out the network you created last week, we will be using that again.

>**Ex. 5.1.4**: Remove all nodes that have zero local clustering coefficient. Print the number of nodes and edges in this new graph. Depending on the regex you used to extract the partnerships you may get varying results, but if your network is smaller than ~60 nodes, that's probably not right.

> **Ex. 5.1.5**: Find the communities in this networks. Visualize it, coloring the nodes according to their label.

>*Hint: If you didn't implement the Louvain method in the bonus exercise above, you will need a package which can do it for you. Go ahead and install `python-louvain` by running `conda install -c auto python-louvain` in a terminal. After installation, import it with `import community`.*

### Week 6

> **Ex. 6.2.1 ** Load the municipalities data. 
- What is the CRS of the GeoDataFrame - what projection does it correspond to? What is the measure of distance? 
- Which three munipalities have the largest area?

> Note: to find the entire area of a municipality which consists of island etc. you can use the `unary_union` method for GeoSeries.

> **Ex. 6.2.2** Select rows in the GeoDataFrame as follows. Make two boolean series as described below and make a combined series which takes the value True if both holds, otherwise False.
> - first: row is True if corresponding the row shape is in the Capital Region or Sealand Region (i.e. `'Region Hovedstaden', 'Region Sjælland'`) 
> - second: row is True if the  the row geometry is ***not*** in Bornholm or nearby (i.e. `'Bornholm', 'Christiansø'`)

> *Hint*: recall that we can check if a series elements are elements in a series using the `isin` method


> **Ex. 6.2.3** Extract the extremum values (min,max) in all dimensions.

> *Hint*: extreme values, i.e. bounds, can be found using `.bounds` on a GeoDataFrame (also works on shapes, GeoSeries) - you further need to compute the global max, min values of these.

> **Ex. 6.2.4** We are now to construct a 500mx500m grid for the bounds of Sealand and around:
- Make a grid of points 500m apart in horizontal and vertical directions that are within the extremum values of Sealand's shape. 


> **Ex. 6.2.5** Compute interpolation of house price for each grid cell as follows:
- Load the pre-structured data with house sales prices for the capital region of Denmark. It is available under the internal course page under files, see [here](https://absalon.instructure.com/courses/24660/files).
- Make a loop over sale_year (2012, 2013):
    - Fit a nearest neighbor regression model to the square meter price (i.e. `price_area` for each year). Set number of neighbors to 25 and radius to 25000        
    - Apply the model to the grid data and assign the estimated price as a column called `pYYYY` where YYYY is the year.
    - Make a new column where you store log10 of house price for the given year and call it`pYYYY_log`
    
> *Hint 1:* sklearn has a regression for k-nearest neighbors

> *Hint 2:* use the Easting and Northing in house prices which correspond to the UTM Z32 North CRS, i.e Denmark. In the dataset they are called 'e' and 'n'.

> **Ex 6.3.6:** Visualize the house data
- For each of these points construct a square polygon assuming that the point is the south west corner of the square. 
- Select all sqaures that intersects with Sealand and nearby islands.
- Plot the grid data for 2012

> *Hint:* Once you have created the grid the following function below may be useful for converting into a GeoDataFrame. You need to specify the column names for your x and y coordinates.

> *Hint 2:* We can select the points that intersect by using a spatial join between the house locations and municipalities.m

In [ ]:
from shapely.geometry import box

dk_crs = {'ellps': 'GRS80', 'no_defs': True, 'proj': 'utm', 'units': 'm', 'zone': 32}

def cell_coords_to_polygons(square_df, x='e', y='n', dist=500, crs=dk_crs):
    '''
    Convert coordinates to squares in a GeoDataFrame.
       
    Parameters
    ----------
    x : str
        Name of the horizontal coordinate (~longitude)            
    y : str
        Name of the vertical coordinate (~latitude)                        
    dist : int or float
        Size of polygons
    crs : dict
        Coordinate Reference System


    Returns
    ----------
    squares_gdf: geopandas.GeoDataFrame
        This table contains squares as geometry
        and the original data.
    '''
    
    def _to_square_polygon(row):
        '''
        This auxiliary function convert a square's lower,left 
        coordinates to a polygon. 
        
        Parameters
        ----------
        row : pandas.Series
            This is a DataFrame row.            
        
        Returns
        ----------
        poly: shapely.Polygon        
        
        '''
        
        d = dist/2
        
        poly = box(row[x]-d, row[y]-d, row[x]+d, row[y]+d)
        
        return poly
    
    # convert to polygons
    square_geoms = gpd.GeoSeries(square_df.apply(_to_square_polygon, axis=1), crs=crs)
    
    # make GeoDataFrame
    square_gdf = gpd.GeoDataFrame(data=square_df, geometry=square_geoms)
    
    return square_gdf

# convert example
df_example = pd.DataFrame([(617288, 6049782),(617288, 6050282)], columns=['e','n'])
gdf_example = cell_coords_to_polygons(df_example)